In [1]:
import copy
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pprint
from scipy.optimize import linprog
from pytest import approx

from mex.simplex.simplex_networks import create_matrix, pivots_col, pivots_row, find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot
from mex.simplex.problem_definition import add_cons, constrain, add_obj, obj, maxz, minz

## Problema prototipo:

$$\min_{x \in \mathbb{R}^2} 2x_1 + 4x_2 + 9x_3 + 3x_4 + x_5 + 3x_6 + 2x_7$$

$$\text{sujeto a: }$$
$$ x_1 \leq 10 $$
$$ x_5 \leq 80 $$
$$ x_1 + x_2 + x_3 = 50$$
$$ -x_1 + x_4 = 40 $$
$$ -x_2 - x_4 + x_5 = 0 $$
$$ -x_3 + x_6 - x_7 = -30 $$
$$ -x_5 - x_6 + x_7 = -60$$

## Resolvemos con scipy

In [2]:
c_net_obj = [2,4,9,3,1,3,2]
A_ub_net_obj = [[1,0,0,0,0,0,0],
            [0,0,0,0,1,0,0]]
b_ub_net_obj = [10,80]
A_eq_net_obj = [[1,1,1,0,0,0,0],
         [-1,0,0,1,0,0,0],
         [0,-1,0,-1,1,0,0],
         [0,0,-1,0,0,1,-1],
         [0,0,0,0,-1,-1,1]]
b_eq_net_obj = [50,40,0,-30,-60]

In [3]:
net_obj = linprog(c_net_obj, A_ub=A_ub_net_obj, b_ub=b_ub_net_obj, A_eq=A_eq_net_obj, b_eq=b_eq_net_obj).fun

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  """Entry point for launching an IPython kernel.


In [4]:
net_coeff_obj = linprog(c_net_obj, A_ub=A_ub_net_obj, b_ub=b_ub_net_obj, A_eq=A_eq_net_obj, b_eq=b_eq_net_obj).x

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  """Entry point for launching an IPython kernel.


## Resolvemos con nuestro paquete

In [5]:
n_var_approx = 7
n_cons_approx = 7

In [6]:
matrix_net_approx = create_matrix(n_var_approx,n_cons_approx)

In [7]:
constrain(matrix_net_approx,'1,1,1,0,0,0,0,E,50')
constrain(matrix_net_approx,'-1,0,0,1,0,0,0,E,40')
constrain(matrix_net_approx,'0,-1,0,-1,1,0,0,E,0')
constrain(matrix_net_approx,'0,0,-1,0,0,1,-1,E,-30')
constrain(matrix_net_approx,'0,0,0,0,-1,-1,1,E,-60')
constrain(matrix_net_approx,'1,0,0,0,0,0,0,L,10')
constrain(matrix_net_approx,'0,0,0,0,1,0,0,L,80')
obj(matrix_net_approx,'2,4,9,3,1,3,2,0')

In [8]:
problem_approx = minz(matrix_net_approx)

In [9]:
net_approx = problem_approx['min']
problem_approx.pop('min')
net_coeff_approx = np.array(list(problem_approx.values()))

In [10]:
assert net_obj == approx(net_approx)
assert np.round(net_coeff_obj,0) == approx(net_coeff_approx)